In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import datetime
import numpy as np

In [7]:
df= pd.read_csv('../input/crimesdistrict/crime_with_coordinates.csv', low_memory=False)
df.head()

,Unnamed: 0,incident_id,case_number,incident_datetime,incident_type_primary,incident_description,clearance_type,address_1,address_2,city,...,parent_incident_type,Census Tract 1,Census Block 1,Census Block Group 1,Neighborhood 1,Police District 1,Council District 1,neighborhood,police_dt,council_dt
0,59596,55236827.0,11-0050386,1910-12-22 00:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,200 Block CRESTWOOD AV,NaN,BUFFALO,...,Theft,NaN,NaN,NaN,NaN,NaN,NaN,North Park,District D,DELAWARE
1,121509,710086379.0,15-0760407,1914-12-01 22:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,100 Block LINCOLN PW,NaN,BUFFALO,...,Theft,NaN,NaN,NaN,NaN,NaN,NaN,Elmwood Bidwell,District D,DELAWARE
2,204751,942663542.0,06-1840984,1951-07-03 19:51:00,UUV,Buffalo Police are investigating this report o...,NaN,200 Block W FERRY ST,NaN,BUFFALO,...,Theft of Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,Upper West Side,District B,NIAGARA
3,146812,146621859.0,13-0730379,1951-12-05 02:20:21,ASSAULT,Buffalo Police are investigating this report o...,NaN,200 Block CAMBRIDGE AV,NaN,BUFFALO,...,Assault,NaN,NaN,NaN,NaN,NaN,NaN,Genesee-Moselle,District E,MASTEN
4,233717,942606470.0,08-3530528,1952-08-30 16:00:00,LARCENY/THEFT,Buffalo Police are investigating this report o...,NaN,200 Block JEFFERSON AV,NaN,BUFFALO,...,Theft,NaN,NaN,NaN,NaN,NaN,NaN,Broadway Fillmore,District C,ELLICOTT


In [9]:
df.shape

(267997, 30)

In [10]:
import datetime
df_police = df[['incident_datetime', 'police_dt']]

In [11]:
df_police.incident_datetime = pd.to_datetime(df_police.incident_datetime)
df_police = pd.DataFrame({'no_of_incidents' : df_police.groupby(
   [ df_police.police_dt, df_police.incident_datetime.dt.date] ).size()}).reset_index()

In [12]:
df_police.tail()

,police_dt,incident_datetime,no_of_incidents
27380,District E,2021-02-15,6
27381,District E,2021-02-16,3
27382,District E,2021-02-17,2
27383,District E,2021-02-18,3
27384,District E,2021-02-19,2


In [13]:
df_police.isna().sum()


police_dt            0
incident_datetime    0
no_of_incidents      0
dtype: int64

In [14]:
#Data Series Start Date	1/2009 (unreliable data before this date)
df_police['year'] = pd.DatetimeIndex(df_police['incident_datetime']).year
df_police= df_police[df_police.year>=2009]

#df['year']= df['year'].astype(str)
#df['hour_of_day']= df['hour_of_day'].astype(str)
df_police.head()

,police_dt,incident_datetime,no_of_incidents,year
995,District A,2009-01-01,17,2009
996,District A,2009-01-02,8,2009
997,District A,2009-01-03,8,2009
998,District A,2009-01-04,6,2009
999,District A,2009-01-05,9,2009


In [15]:
df_police['year'].unique()


array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021])

In [16]:
df_police.nunique()

police_dt               5
incident_datetime    4433
no_of_incidents        34
year                   13
dtype: int64

#### create dummies variables district

In [17]:
dummy_district = pd.get_dummies(df_police['police_dt'])

dummy_district.columns = df_police['police_dt'].unique()

df_police = pd.concat([df_police,dummy_district], axis=1)


In [18]:
df_police.drop(['incident_datetime', 'police_dt'] ,axis= 1,  inplace=True)

In [19]:
df_police.head()

,no_of_incidents,year,District A,District B,District C,District D,District E
995,17,2009,1,0,0,0,0
996,8,2009,1,0,0,0,0
997,8,2009,1,0,0,0,0
998,6,2009,1,0,0,0,0
999,9,2009,1,0,0,0,0


## Kcross validation

In [20]:
from sklearn.model_selection import KFold # import KFold
from sklearn.metrics import accuracy_score #To calculate accuracy
from sklearn.metrics import mean_squared_error

In [21]:
# to classify the count of incident int number
def my_ceil(predictions):
    for i in range(len(predictions)):
        if predictions[i]%1 <= +0.5:
            predictions[i] = int(predictions[i])
        else:
            predictions[i] = int(predictions[i]) + 1
    return predictions

def fun_cross_validation(df, n_kf, model):
    accuracy = []
    kf = KFold(n_splits = n_kf)
    for train_index, test_index in kf.split(df):
        #train part
        X_train = np.array(df.drop('no_of_incidents', axis = 1))[train_index]
        y_train = np.array(df.no_of_incidents)[train_index]
        #test part
        X_test = np.array(df.drop('no_of_incidents', axis = 1))[test_index]
        y_test = np.array(df.no_of_incidents)[test_index]
        #model fit
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred = my_ceil(np.array(y_pred))
        #model evaluation
        accuracy.append(accuracy_score(y_pred, y_test))
    return np.average(accuracy), mean_squared_error(y_pred, y_test)


## Models 

### Xgboost

In [22]:
import xgboost

In [23]:
model_xg = xgboost.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.05,
                          alpha = 10,base_score= 0.4, booster='gbtree', n_estimators = 1000)

In [24]:
fun_cross_validation(df_police, 10, model_xg)

(0.10746606334841628, 12.034841628959276)

# Pipeline

In [27]:
X=  df_police.drop('no_of_incidents', axis = 1)
y = df_police.no_of_incidents

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y , test_size= 0.2, random_state= 42)

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#SimpleImputer fill any missing values 
#Scaler numeric transformer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

#One hot encoder to transform categorial values into integers.

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [32]:
#Select les columns numeric 
#Select les columns categoric 
integer_features = list(X.columns[X.dtypes == 'int64'])
continuous_features = list(X.columns[X.dtypes == 'float64'])
categorical_features = list(X.columns[X.dtypes == 'object'])
numeric_features = integer_features + continuous_features 


from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

## Model selection

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(), 
    LinearDiscriminantAnalysis()
    ]
pipes= []
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)  
    pipes.append(pipe) 
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))


KNeighborsClassifier(n_neighbors=3)
model score: 0.074
SVC(C=0.025, probability=True)
model score: 0.088
DecisionTreeClassifier()
model score: 0.093
RandomForestClassifier()
model score: 0.090
AdaBoostClassifier()
model score: 0.085
